# 함수, 반복자, 제네레이터

함수형 프로그래밍의 핵심은 순수 함수를 사용해 입력 정의력의 값을 출력 치역의 값으로 바꾸는 것이다. 순수 함수에는 부수 효과가 없으며, 이러한 순수 함수를 파이썬으로 구현하는 것은 상대적으로 쉽다.

부수 효과가 없다는 것은 계산에 필요한 상태를 유지하기 윈한 변수 대입에 따른 의존 관계를 줄일 수 있다는 의미다. 파이썬 언어에서 대입문을 아예 없앨 수는 없지만, 상태가 있는 객체에 대한 의존성을 줄일 수는 있다. 

제네레이터와 제네레이터 식이 객체의 컬렉션을 가지고 작업하기 위한 방법이기 때문에 이에 대해 살펴본다. 모든 제네레이터 식을 재귀로 바꿀 때는 몇 가지 주변적인 문제가 생긴다. 파이썬은 재귀 깊이에 제한을 두며, 자동으로 TCO하지 않는다. 우리는 제네레이터 식을 사용해 직접 재귀를 최적화해야만 한다.

함수형 파이썬을 작성하려면 튜플과 변경할 수 있는 컬렉션에 초점을 맞출 필요가 있다.

## 순수 함수 작성하기

순수 함수에는 부수 효과가 없다. 즉 변수의 전역적인 상태를 변경하는 일이 결코 없다. global 문을 ㅅ용하지 안흔다면 이러한 목표를 거의 충족할 수 있다. 

순수 함수의 두 가지 측면을 보장할 수 있는 방식을 몇 가지 살펴본다. 자유 변수를 사용해 파이썬의 전역에 있는 값을 참조하면 매개변수를 적절히 사용해 처리할 수 있다. 

상태가 있는 파이썬 내부 객체는 많다. file 클래스의 인스턴스나 모든 파일과 유사한 객체들은 자주 사용되는 상태가 있는 객체의 예다. 
상태가 있는 파일 객체 사용을 잘 정의된 영역 안으로 제한할 수 있는 with문을 사용해야 한다.

전역 파일 객체, 전역 데이터베이스 연결 등을 피하고 그와 관련 있는 상태를 피해야 한다. 전역 파일 객체는 열린 파일을 처리하는 경우에 매우 흔하게 사용하는 패턴이다. 

In [1]:
def open(iname, oname):
    global ifile, ofile
    ifile = open(iname, "r")
    ofile = openr(oname, "w")

다양한 다른 함수들이 ifile이나 ofile 변수를 사용할 수 있다. 이러한 함수들은 모두 어디선가 애플리케이션에서 사용하도록 열어둔, 제대로 된 global 파일을 참조할 수 있으리라는 기대를 하고 있다.

이는 그리 좋은 설계가 아니며, 그러한 방식을 피해야만 한다. 파일은 함수에 제공되는 매개변수여야 하며, 열린 파일은 with문으로 감싸서 상태에 따른 동작을 제대로 처리해야 한다.

이러한 설계 패턴은 데이터베이스에도 적용할 수 있다. 일반적으로 데이터베이스 연결 객체를 애플리케이션의 함수에 형식 인자로 제공할 수 있다. 이는 몇몇 유명한 웹 프레임워크가 전역 데이터베이스 연결을 활용해 애플리케이션 전체에서 그 연결을 투명하게 활용하려는 노력을 기울이는 것과는 상반된 방식이다. 이와 더불어 다중 스레드 웹 서버는 단일 데이터베이스 연결을 공유하는 것으로부터 성능상의 이익을 얻지 못할 수도 있다. 

### 일급 계층 객체인 함수

파이썬의 함수가 일급 계층 객체라는 점에 놀라지 말아야 한다. 함수는 파이썬에서 몇 가지 애트리뷰트가 있는 객체이다. 참조 매뉴얼을 찾아보면 함수에 적용할 수 있는 몇 가지 특별한 멤버 이름을 알 수 있다. 함수가 애트리뷰트가 있는 객체이기 때문에 \__name__ 애트리뷰트를 사용해 함수이름이나 docstring을 뽑아낼 수 있다. 또한 함수의 본문을 \__code__ 애트리뷰트를 사용해 가져올 수 있다. 컴파일 언어에서는 소스 코드 정보를 유지해야 하기 때문에 이러한 인트로스팩션은 상대적으로 복잡하다. 하지만 파이썬에서는 꽤 간단하다.

함수를 변수에 대입하거나, 함수를 인자로 넘기거나, 함수를 값으로 반환할 수 있다. 이러한 기법을 사용하면 고차 함수를 쉽게 작성할 수 있다.

함수가 객체이기 때문에 파이썬은 함수형 프로그래밍이 되는 데 필요한 여러 기능을 이미 갖추고 있다.

이와 아울러 호출 가능한 객체를 사용해 함수를 만들 수도 있다. 호출 가능한 객체도 일급 계층 객체다. 심지어 호출 가능 클래스 정의를 고차 함수라고 생각할 수도 있다. 호출 가능 객체에서 \__init__() 메서드를 사용하는 방식을 신중하게 생각할 필요가 있다. 즉, 상태가 클래스 변수를 피해야 한다. 하 가지 일반적인 응용 방법은 \__init__() 메서드를 사용해 전략 디자인 패턴에 부합하는 객체를 만드는 것이다.

전략 디자인 패턴을 따르는 클래스는 알고리즘이나 알고리즘의 일부를 제공하는 다른 객체에 의존한다. 이 패턴은 알고리즘의 자세한 부분을 클래스 안에 컴파일해 넣는 대신, 실행 시점에 알고리즘의 세부 사항을 주입할 수 있게 해준다.

다음은 내장 전략 객체가 있는 호출 가능한 객체를 보여준다.

In [4]:
import collections

class Mersennel(collections.Callable):
    def __init__(self, algorithm):
        self.pow2 = algorithm
    
    def __call__(self, arg):
        return self.pow2(arg) -1

이 클래스는 \__init__()를 사용해 다른 함수에 대한 참조를 저장한다. 하지만 아무런 상태가 있는 인스턴스 변수를 만들지 않는다. 이러한 객체에 주어지는 함수는 주어진 인자만큼 2를 거듭제곱한다. 이 클래스에 끼워 넣을 수 있는 세 가지 후보 객체는 다음과 같다.

In [6]:
def shifty(b):
    return 1 << b
    
def multy(b):
    if b == 0: return 1
    return 2*multy(b-1)

def faster(b):
    if b == 0: return 1
    if b % 2 == 1: return 2*faster(b-1)
    t = faster(b//2)
    return t

shifty 함수는 비트 왼쪽 시프트 연산을 사용해 2의 거듭제곱을 계산한다. multy() 함수는 단순한 재귀 곱셈을 사용한다. faster() 함수는 분할 정복 전략을 사용해 b번이 아니라 $log_2(b)$번의 곱셈을 수행한다.

알고리즘 전략을 내장한 Mersennel 클래스의 인스턴스는 다음과 같이 만들 수 있다.

In [7]:
m1s = Mersennel(shifty)
m1m = Mersennel(multy)
m1f = Mersennel(faster)

이는 결과는 같지만, 서로 다른 알고리즘을 사용하는 다양한 함수를 만드는 방법을 보여준다.

In [8]:
m1s(10)

1023

In [12]:
m1m(10)

1024

In [10]:
m1f(10)

3

* 파이썬은 $M_{89} = 2^{89} -1 까지 계산을 허용한다. 이는 파이썬의 재귀호출 깊이 제한값은 그 근처에도 가지 못할 만큼 큰 값이다. 이 값은 27자릿수인 상당히 큰 소수다.

### 문자열 사용하기 

파이썬의 문자열은 변경 불가능하기 때문에 함수형 프로그래밍 객체의 좋은 예라고 할 수 있다. 파이썬의 string 모듈에는 많은 메서드가 들어 있고, 그들 모두는 결과로써 새로운 문자열을 내놓는다. 이러한 메서드는 부수 효과가 없는 순수 함수다.

전위 방식인 대부분의 함수와 달리 string 메서드 함수들을 사용하는 구문은 후위 방식이다. 이는 복잡한 문자열 연산이 일반적인 함수와 혼합되는 경우, 가독성이 떨어진다는 의미다.

웹 페이지에서 데이터를 긁어오는 경우, 긁어온 문자열에 일련의 변환을 적용해 구분 기호 등을 제거하고 애플리케이션의 다른 부분에서 사용할 수 있는 Decimal 객체를 반환하는 정리 함수를 만들 수 있다. 그 과정에서 전위 연산자와 후위 연산자 호출이 필요하다.

In [13]:
from decimal import *

def clean_decimal(text):
    if text is None: return text
    try:
        return Decimal(text.replace("$", "").replace(",", ""))
    except InvalidOperation:
        return text

이 함수는 $와 , 문자열을 제거하기 위해 replace를 두 번 호출한다. 

이를 더욱 일관성 있게 만들려면 다음과 같이 string의 메서드 함수를 처리하는 전위 연산자 함수를 정의해야 한다.

In [14]:
def replace(data, a, b):
    return data.replace(a, b)

이러한 함수가 있으면 좀더 일관성 있는 전위 연산자 형태의 Decimal(replace(replace(text, "$", ""), ",", "")) 이라는 구문을 사용할 수 있다. 이 함수는 기존 인자의 위치를 재배열하여 새로운 기법을 사용할 수 있게 했다.

이러한 일관성이 전위와 후위 연산을 섞어 사용하는 것에 비해 얼마나 큰 개선인지는 분명하지 않아 보인다.  인자가 많은 함수의 문제는 인자들이 식의 여기 저기에 나타날 수 있다는 점이다.

좀 더 나은 접근 방식은 다음과 비슷하게 구분 기홀ㄹ 정리해주는 전위 연산자 방식의 함수를 정의하는 것이다.

In [15]:
def remove(str, chars):
    if chars: return remove(str.replace(chars[0], ""), chars[1:])
    return str

이 함수는 재귀적으로 char 변수에 있는 글자들을 str에서 제거한다. 이를 Decimal(remove(text, "$,"))와 같이 사용하면 문자열을 정리하는 우리의 목적을 더욱 잘 드러낼 수 있다.

### tuple과 namedtuple 사용하기

tuple은 변경 불가능한 객체다. 이 또한 함수형 프로그래밍에 적합한 객체에 대한 훌륭한 예다. 파이썬의 tuple은 소수의 메서드 함수만 제공하기 때문에 거의 대부분의 작업은 전위 문법을 사용한 함수를 통해 이뤄진다. 튜플을 사용하는 예는 다양하지만, 특히 튜플의 리트스, 튜플의 튜플, 튜플들을 만들어 내는 제너레이터 등을 자주 사용한다.

물론 namedtuple은 튜플에 필수적인 기능을 하나 더 추가한다. 즉, 인덱스 대신 이름을 사용할 수 있게 해준다. namedtuple을 활용하면 데이터를 취합한 객체를 만들 수 있다. 이를 통해 상태가 없는 객체를 기반으로 하는 순수 함수를 작성하면서도 깔끔하게 객체와 비슷한 용기에 데이터를 담을 수 있다.

값의 모음인 경우에는 대부분 튜플을 사용할 것이다. 값을 하나만 사용하거나 단순히 두 값을 필요로 하는 경우에는 보통 함수에 이름이 정해진 매개변수를 전달하는 방식을 사용할 것이다. 하지만 컬렉션을 사용하는 경우에는 튜플의 반복자나 이름 있는 튜플의 반복자를 사용해야 할 수도 있다.

tuple이나 namedtuple 중 어느 쪽을 사용할 것인지는 순전히 평의의 문제다. (수, 수, 수) 형태의 3-튜플의 시퀀스를 사용하면서, 3-튜플의 각 원소가 적색, 녹색, 청색을 표현한다고 가정할 수도 있다.

3-튜플에서 값을 가져오기 위해 ㅏ음과 같은 함수를 사용할 수 있다.

In [16]:
red = lambda color: color[0]
green = lambda color: color[1]
blue = lambda color: color[2]

물론 다음과 같이 이름 있는 튜플을 사용할 수도 있다.

In [19]:
  from collections import namedtuple

Color = namedtuple("Color", ("red", "green", "blue", "name"))

이렇게 하면 red(item) 대신 item.red를 사용할 수 있다.

튜플을 함수형 프로그래밍에서 활용하는 것은 주로 튜플의 반복 가능 클래스를 근간으로 한다. 이러한 기법 중 몇 가지를 좀 더 자세히 살펴본다. 

### 제너레이터 식 사용하기

몇 가지 복잡한 제너레이터 기법을 소개한다. 

파이썬 문법의 일부를 여기서 설명할 것이다. 이는 list나 dict 내장을 통해 list나 dict 리터럴을 만들어 내는 제너레이터 식에서 흔히 사용하는 것이다. 리스트 디스플레이(display) 또는 내장(comprehension)은 제너레이터 식을 사용하는 방법 중 하나일 뿐이다. 우리는 디스플레이 밖에서 제너레이터 식을 사용하는 것과 디스플레이 안에서 사용하는 것을 구분할 수 있다. 하지만 그렇게 한다고 해서 얻을 수 있는 식일은 없다. 

리스트 디스플레이에는 [x**2 for x in range(10)];과 같은 리터럴 문법이 들어간다. 예를 들어 리스트 내장은 내부에 포함된 제너레이터 식으로부터 리스트 객체를 생성하다. 

주어진 수의 소인수를 구한다. 주어진 x가 짝수라면 2를 내놓고, 그 후에 x/2의 소인수를 내놓는다.

In [22]:
import math

def pfactorsl(x):
    if x % 2 == 0:
        yield 2
        if x //2 > 1:
            yield from pfactorsl(x//2)
        return
    for i in range(3, int(math.sqrt(x) +.5) +1, 2):
        if x % i == 0:
            yield i
            if x // i > 1:
                yield from pfactorsl(x//i)
            return
    yield x

In [27]:
p_10 = pfactorsl(10)


In [28]:
next(p_10)

2

In [29]:
next(p_10)

5

In [30]:
next(p_10)

StopIteration: 

소인수 중에 2를 따로 처리하여 전체 반복 회수를 절반으로 줄인다. 2를 제외한 모든 소수는 홀수이기 때문이다.

재귀 외에도 중요한 for루프를 하나 더 사용했다. 루프를 통해 인수가 1,000개인 경우까지 수비게 처리할 수 있다. 이 수는 최소한 2^1000, 즉 300자리 숫자 이상의 크기를 가지고 있다. 

결과적으로 꼬리재귀 최적화를 수행한 셈이다. 재귀호출은 3부터 루트 x까지를 센다. for 루프는 그 범위 안에 있는 모든 수를 하나하나 검토하는 재귀호출을 하지 않고도 원하는 바를 이룰 수 있게 해준다.

* 재귀적인 제너레이터 함수에서는 return 문을 사용할 때 조심해야 한다. 다음과 같은 문장을 사용하면 안 된다.
    
    return recursive_iters(args)

* 이와 같이 하면 제너레이터 객체를 반환하기만 하고, 만들어진 값을 반환하도록 함수를 평가하지는 않는다. 따라서 다음 두 가지 방식을 사용해야 한다.

    for result in recursive_iters(args):
        yield result
        
    yield from recursive_iter(args)

이 코드에 대한 대안으로, 더 순수한 재귀를 사용하는 것을 들 수 있다.

In [31]:
def pfactorsr(x):
    def factor_n(x, n):
        if n*n > x:
            yield x
            return 
        if x % n == 0:
            yield n
            if x // n > 1:
                yield from factor_n(x//n, n)
        else:
            yield from factor_n(x, n+2)
    
    if x % 2 == 0:
        yield 2
        if x // 2 > 1:
            yield from pfactorsr(x//2)
        return
    yield from factor_n(x, 3)

내부적인 재귀함수 factor_n()을 정의해 3 <= x <= 루트 x 범위에서 인수 n을 검사했다. 만약 후보 인수인 n이 이 범위에 들지 못하면, x는 소수다. 그렇지 않다면, n이 x의 인수인지 살펴본다. 인수인 경우에는 n을 내놓고, x/n의 모든 인수를 내놓는다. n이 인수가 아니라면, 이 함수를 n+2를 사용해 재귀적으로 계산한다. 

* 여기 있는 순수 재귀함수는 오직 4,000,000 정도까지만 소인수를 계산할 수 있다. 이를 넘어가는 경우에는 파이썬의 재귀호출 깊이 제한에 도달한다.

### 제너레이터의 한계

제너레이터 식이나 함수에는 몇 가지 한계가 있는데, 다음 코드는 이러한 한계를 보여준다.

In [32]:
pfactorsl(1560)

<generator object pfactorsl at 0x066241B0>

In [33]:
list(pfactorsl(1560))

[2, 2, 2, 3, 5, 13]

In [34]:
len(pfactorsl(1560))

TypeError: object of type 'generator' has no len()

첫 번째 예에서는 제너레이터 식이 엄격하지 않다는 것을 볼 수 있다. 지연 계산을 수행하며, 해당 제너레이터 함수를 소비하기 전까지는 적절한 값이 들어있지 않다.

두 번째 예에서는 제너레이터 식으로부터 리스트 객체를 구체화했다. 출력을 살펴보거나 단위 테스트 케이스를 작성하는 경우에는 이러한 기능이 유용하다.

시 번째 예에서는 제너레이터 함수의 한 가지 한계인 len()이 없다는 점을 알 수 있다.

제너레이터 함수의 다른 한계는 오직 한 번밖에 쓸 수 없다는 것이다. 

In [35]:
result = pfactorsl(1560)

In [36]:
sum(result)

27

In [37]:
sum(result)

0

첫 번째 sum에서는 제너레이터를 모두 계산했다. 두 번째로 sum에서는 제너레이터가 비어있다. 제너레이터가 만들어 내는 값을 오직 한 버만 소비할 수 있다.

파이썬 제너레이터는 상태가 있다. 일부 함수형 프로그래밍에 있어 제너레이터가 매우 유용하지만, 그렇게 완전하지는 않다. itertools.tree() 메서드를 사용하면 한 번밖에 사용하지 못하는 한계를 넘어설 수 있다. 

In [38]:
import itertools
def limits(iterable):
    max_tee, min_tee = itertools.tee(iterable, 2)
    return max(max_tee), min(min_tee)

In [40]:
limits(pfactorsl(1560))

(13, 2)

### 제너레이터 식 조합하기

제너레이터 하무를 결합하는 일반적인 방법은 합성 함수를 만드는 경우다. (f(x) for x in range())를 계산하는 제너레이터가 있다고 가정해보자. 이때 g(f(x))를 계산하고 싶다면, 두 제너레이터를 결합하는 데에는 여러 가지 방법이 있다.

g_f_x = (g(f(x)) for x in range())

기술적으로 문제는 없지만, 이는 재사용을 막는다. 식을 재사용하기 보다는 코드를 재작성하기 때문이다. 

이를 해결하기 위해 단순한 치환으로 조합을 할 수 있다. 

f_x = (f(x) for x in range())

g_f_x = (g(y) for u in f_x)

### 제너레이터 함수를 사용해 원자료 정리하기

탐색적 자료 분석에서 부각되는 작업 중 하나는 원자료를 정리하는 것이다. 이러한 과정은 종종 몇 가지 스칼라 함수를 입력 데이터의 여러 부분에 적용하여 사용하기 쉬운 자료 집합을 만들어 내는 연산을 합성하여 이뤄진다.

In [5]:
import csv
def row_iter(source):
    return csv.reader(source, delimiter="\t")

In [8]:
with open("Anscombe.txt") as source:
    print(list(row_iter(source)))

[["Anscombe's quartet"], ['I', 'II', 'III', 'IV'], ['x', 'y', 'x', 'y', 'x', 'y', 'x', 'y'], ['10.0', '8.04', '10.0', '9.14', '10.0', '7.46', '8.0', '6.58'], ['8.0', '6.95', '8.0', '8.14', '8.0', '6.77', '8.0', '5.76'], ['13.0', '7.58', '13.0', '8.74', '13.0', '12.74', '8.0', '7.71'], ['9.0', '8.81', '9.0', '8.77', '9.0', '7.11', '8.0', '8.84'], ['11.0', '8.33', '11.0', '9.26', '11.0', '7.81', '8.0', '8.47'], ['14.0', '9.96', '14.0', '8.10', '14.0', '8.84', '8.0', '7.04'], ['6.0', '7.24', '6.0', '6.13', '6.0', '6.08', '8.0', '5.25'], ['4.0', '4.26', '4.0', '3.10', '4.0', '5.39', '19.0', '12.5'], ['12.0', '10.84', '12.0', '9.13', '12.0', '8.15', '8.0', '5.56'], ['7.0', '4.82', '7.0', '7.26', '7.0', '6.42', '8.0', '7.91'], ['5.0', '5.68', '5.0', '4.74', '5.0', '5.73', '8.0', '6.890']]


이 방식이 지니고 있는 문제는 결과 반복자가 돌려주는 첫 세 줄은 우리가 원하는 자료가 아니라는 것이다.

이 세 줄을 걸러내야 한다. 다음은 세 줄을 깔끔하게 가져오고, ㄴ머지 줄만 포함하고 있는 반복자를 반환하는 함수다.

In [9]:
def head_split_fixed(row_iter):
    title = next(row_iter)
    assert len(title) == 1 and title[0] == "Anscombe's quartet"
    heading = next(row_iter)
    assert len(heading) == 4 and heading == ["I", "II", "III", "IV"]
    columns = next(row_iter)
    assert len(columns) == 8 and columns == ['x', 'y', 'x', 'y', 'x', 'y', 'x', 'y']
    return row_iter

In [10]:
with open("Anscombe.txt") as source:
    print(list(head_split_fixed(row_iter(source))))

[['10.0', '8.04', '10.0', '9.14', '10.0', '7.46', '8.0', '6.58'], ['8.0', '6.95', '8.0', '8.14', '8.0', '6.77', '8.0', '5.76'], ['13.0', '7.58', '13.0', '8.74', '13.0', '12.74', '8.0', '7.71'], ['9.0', '8.81', '9.0', '8.77', '9.0', '7.11', '8.0', '8.84'], ['11.0', '8.33', '11.0', '9.26', '11.0', '7.81', '8.0', '8.47'], ['14.0', '9.96', '14.0', '8.10', '14.0', '8.84', '8.0', '7.04'], ['6.0', '7.24', '6.0', '6.13', '6.0', '6.08', '8.0', '5.25'], ['4.0', '4.26', '4.0', '3.10', '4.0', '5.39', '19.0', '12.5'], ['12.0', '10.84', '12.0', '9.13', '12.0', '8.15', '8.0', '5.56'], ['7.0', '4.82', '7.0', '7.26', '7.0', '6.42', '8.0', '7.91'], ['5.0', '5.68', '5.0', '4.74', '5.0', '5.73', '8.0', '6.890']]


이 함수는 반복 가능 객체에서 세 줄을 없앤다. 이때 각 줄에 원하는 내용이 있는지를 assert로 확인한다. 그러한 정보가 없다면 해당 파일이 오염됐거나 우리가 잘못된 파일을 대상으로 분석을 시도하고 있다는 것을 나타내는 징후일 것이다.

한 반복자를 다른 반복자의 결과에 적용했을 뿐이다. 실제로 이는 합성 함수를 정의하는 것과 같다. 

### list, dict, set 사용하기

파이썬에서 list 디스플레이는 제너레이터를 실체화하기 위한 구문을 제공한다. 단지 []만을 추가하면 된다. 이는 제너레이터 식과 리스트 내장을 구분해주는 요소다. 다음은 실체화하는 것을 보여준다.

In [11]:
range(10)

range(0, 10)

In [12]:
[range(10)]

[range(0, 10)]

In [13]:
[x for x in range(10)]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [14]:
list(range(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

첫 번째 예는 제너레이터 함수다.

* range 함수는 지연 계산 함수다. 각 값을 모두 사용하는 문맥에 도달하지 않는 경우, 그 함수는 10개의 값을 모두 만들지 않을 것이다.

두 번째 예는 단일 제너레이터 함수로 구성한 리스트를 보여준다. 

세 번째 예는 제너레이터 함수 안에 포함된 제너레이터 식으로부터 만들어지는 list 내장을 보여준다. 함수 range(10)은 제너레이터 식인 x for x in range(10)에 의해 평가된다. 결과 값은 list 객체에 수집된다.

* list(range(10)) 함수는 제너레이터 식을 평가한다. [range(10)] 리스트 리터럴은 제너레이터 함수를 평가하지 않는다.

튜플을 실체화하려면 tuple() 함수를 사용해야만 한다. 

데이터 정리 예제에서 합성 함수를 사용해 네 튜플의 리스트를 만들었다. 그 함수는 다음과 같다.

In [16]:
with open("Anscombe.txt") as source:
    data = head_split_fixed(row_iter(source))
    print(list(data))

[['10.0', '8.04', '10.0', '9.14', '10.0', '7.46', '8.0', '6.58'], ['8.0', '6.95', '8.0', '8.14', '8.0', '6.77', '8.0', '5.76'], ['13.0', '7.58', '13.0', '8.74', '13.0', '12.74', '8.0', '7.71'], ['9.0', '8.81', '9.0', '8.77', '9.0', '7.11', '8.0', '8.84'], ['11.0', '8.33', '11.0', '9.26', '11.0', '7.81', '8.0', '8.47'], ['14.0', '9.96', '14.0', '8.10', '14.0', '8.84', '8.0', '7.04'], ['6.0', '7.24', '6.0', '6.13', '6.0', '6.08', '8.0', '5.25'], ['4.0', '4.26', '4.0', '3.10', '4.0', '5.39', '19.0', '12.5'], ['12.0', '10.84', '12.0', '9.13', '12.0', '8.15', '8.0', '5.56'], ['7.0', '4.82', '7.0', '7.26', '7.0', '6.42', '8.0', '7.91'], ['5.0', '5.68', '5.0', '4.74', '5.0', '5.73', '8.0', '6.890']]


이를 유용하게 사용하려면 약간의 처리를 추가해야 한다. 첫째, 8개의 튜플 중에서 10쌍을 골라낼 필요가 있다. 다음과 같은 함수를 통해 가능하다.

In [17]:
from collections import namedtuple

Pair = namedtuple("Pair", ("x", "y"))
def series(n, row_iter):
    for row in row_iter:
        yield Pair(*row[n*2:n*2+2])

이 함수는 0과 4 사이의 번호를 기준으로 인접한 열을 선택한다. 두 열로부터 namedtuple 객체를 만든다. 이름을 붙이면 각 줄에서 x와 y를 선택할 수 있다.

이제 튜플의 튜플 컬렉션을 다음과 같이 만들 수 있다.

In [22]:
global_sample_I = None
with open("Anscombe.txt") as source:
    data = tuple(head_split_fixed(row_iter(source)))
    sample_I = tuple(series(0, data))
    sample_II = tuple(series(1, data))
    sample_III = tuple(series(2, data))
    sample_IV = tuple(series(3, data))
    global_sample_I = sample_I

마지막 작업은 누적했던 문자열로부터 적절한 수를 만들어 내어 통계적 요약 값을 계산할 때 쓸 수 있게 만드는 것이다. float() 변환 함수를 맨 마지막 단계로 적용할 수 있다. 

다음은 float 함수를 쓰는 방법을 보여준다.

In [24]:
mean = sum(float(pair.y) for pair in global_sample_I) / len(global_sample_I)

In [25]:
print(mean)

7.500909090909093


In [27]:
for subset in sample_I, sample_II, sample_III, sample_IV:
    print(subset)
    mean = sum(float(pair.y) for pair in subset) / len(subset)
    print(mean)

(Pair(x='10.0', y='8.04'), Pair(x='8.0', y='6.95'), Pair(x='13.0', y='7.58'), Pair(x='9.0', y='8.81'), Pair(x='11.0', y='8.33'), Pair(x='14.0', y='9.96'), Pair(x='6.0', y='7.24'), Pair(x='4.0', y='4.26'), Pair(x='12.0', y='10.84'), Pair(x='7.0', y='4.82'), Pair(x='5.0', y='5.68'))
7.500909090909093
(Pair(x='10.0', y='9.14'), Pair(x='8.0', y='8.14'), Pair(x='13.0', y='8.74'), Pair(x='9.0', y='8.77'), Pair(x='11.0', y='9.26'), Pair(x='14.0', y='8.10'), Pair(x='6.0', y='6.13'), Pair(x='4.0', y='3.10'), Pair(x='12.0', y='9.13'), Pair(x='7.0', y='7.26'), Pair(x='5.0', y='4.74'))
7.500909090909091
(Pair(x='10.0', y='7.46'), Pair(x='8.0', y='6.77'), Pair(x='13.0', y='12.74'), Pair(x='9.0', y='7.11'), Pair(x='11.0', y='7.81'), Pair(x='14.0', y='8.84'), Pair(x='6.0', y='6.08'), Pair(x='4.0', y='5.39'), Pair(x='12.0', y='8.15'), Pair(x='7.0', y='6.42'), Pair(x='5.0', y='5.73'))
7.500000000000001
(Pair(x='8.0', y='6.58'), Pair(x='8.0', y='5.76'), Pair(x='8.0', y='7.71'), Pair(x='8.0', y='8.84'), 

pair.y는 pair[1]을 사용하는 것보다 명확하다.

메모리 사용을 줄이기 위해(성능은 향상됨) 제너레이터 식과 함수를 더 선호한다. 반복자를 오직 한 번만 사용할 수 있기 때문에 떄로는 컬렉션을 튜플로 실체화시켜야 할 수도 있다. 실체화는 메모리와 시간이 소요되기 때문에 주의를 기울여야 한다.

### 상태가 있는 매핑 사용하기

파이썬은 dict 클래스에 포함된 여러 매핑과 collections 모듈에 정의된 여러 매핑 등과 같이 몇 가지 상태가 있는 컬렉션을 제공한다.

파이썬을 사용해 함수형 프로그래밍 기법을 배우려는 목적 아래 매핑을 사용하는 방법은 두 가지가 있다. 첫 번째는 매핑을 누적시키는 상태가 있는 딕셔너리로 사용하는 방식이고, 두 번째는 고정시킨(frozen) 딕셔너리다. 

점진적으로 맵을 만드는 것은 메모이제이션 시에 유용하다.

